In [1]:
import pandas as pd

import glob
import gc

In [2]:
files = glob.glob('data/COVID_vs_NON_regional/2020*.csv')

In [3]:
len(files)

279

In [4]:
u_cols = ['cl_fst_serv_dt', 'invn_ctl_nbr', 'clm_hc_diag_code1',
       'clm_hc_diag_code2', 'clm_hc_diag_code3', 'clm_hc_diag_code4',
       'clm_hc_diag_code5', 'cov_19', 'cl_proc_cd']

u_colsi = ['cl_fst_serv_dt', 'invn_ctl_nbr', 'clm_hc_diag_code1', 'cl_proc_cd', 'cov_19']


In [5]:
dfs = []
print('Loading Data')
for f in files:
    if int(f[32]) >= 4:
        try:
            df = pd.read_csv(f, parse_dates=['cl_fst_serv_dt'], usecols=u_cols)
        except:
            df = pd.read_csv(f, parse_dates=['cl_fst_serv_dt'], usecols=u_colsi)
        df = df[(df['cl_fst_serv_dt'] >= '2020-04-01') & (df['cl_fst_serv_dt'] <= '2020-05-10')]
        dfs.append(df)
        del df
        gc.collect()
    else:
        pass

Loading Data


In [ ]:
df = pd.concat(dfs)

/usr/local/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [ ]:
del dfs
gc.collect()

In [ ]:
files = glob.glob('data/COVID_vs_NON_member_id/*.csv')

In [ ]:
files

In [ ]:
dfs = []
print('Loading members')
for f in files:
    if int(f[33]) >= 4:
        df2 = pd.read_csv(f, parse_dates=['cl_fst_serv_dt'])
        df2 = df2[(df2['cl_fst_serv_dt'] >= '2020-04-01') & (df2['cl_fst_serv_dt'] <= '2020-05-10')]
        dfs.append(df2)
        del df2
        gc.collect()
    else:
        pass

In [ ]:
df_member = pd.concat(dfs)

In [ ]:
del dfs
gc.collect()

In [ ]:
df_both = pd.merge(left=df, right=df_member, on='invn_ctl_nbr')

In [ ]:
df_both['cl_fst_serv_dt_x'] = pd.to_datetime(df_both['cl_fst_serv_dt_x'], errors='coerce') 

In [ ]:
df_both['month'] = df_both['cl_fst_serv_dt_x'].dt.month

In [ ]:
len(df_both)

In [ ]:
df_both['flagged'] = 0

In [ ]:
len(df_both)

In [ ]:
df_both.sort_values(by='cl_fst_serv_dt_y', inplace=True)

In [ ]:
df_both.head()

In [ ]:
len(df_both.drop_duplicates(subset=['clm_patient_id']))

In [ ]:
df_conf_mem = df_both.drop_duplicates(subset=['clm_patient_id'])

In [ ]:
df_conf_mem = df_conf_mem[['invn_ctl_nbr', 'clm_patient_id']]

In [ ]:
len(df)

In [ ]:
len(df_member)

In [ ]:
len(df_conf_mem)

In [ ]:
df_m_b = pd.merge(left = df, right = df_member, on = 'invn_ctl_nbr')

In [ ]:
df_m_b.head()

In [ ]:
len(df_m_b)

In [ ]:
df_c_m_c = pd.merge(left = df_m_b, right = df_conf_mem, on = 'clm_patient_id') 

In [ ]:
len(df_c_m_c)

In [ ]:
df_c_m_c.head()

In [ ]:
len(df_c_m_c.dropna(subset=['invn_ctl_nbr_x']))

In [ ]:
df_c_m_c['clm_patient_id'].value_counts()

In [ ]:
df_c_m_c = df_c_m_c.sort_values(by=['clm_patient_id','cl_fst_serv_dt_x'])

In [ ]:
mems = df_conf_mem['clm_patient_id'].unique()

In [ ]:
d = dfm['cl_fst_serv_dt_x'][dfm['cov_19']=='Confirmed'].min()

In [ ]:
d2 = dfm['cl_fst_serv_dt_x'].iloc[10]

In [ ]:
d2 - d

In [ ]:
d

In [ ]:
dfs = []

for i, m in enumerate(mems):
    print(str(i) + '/' + str(len(mems)))
    dfm = df_c_m_c[df_c_m_c['clm_patient_id'] == m]
    d = dfm['cl_fst_serv_dt_x'][dfm['cov_19']=='Confirmed'].min()
    dfm = dfm[(dfm['cl_fst_serv_dt_x']<= d) & (dfm['cov_19']!='Confirmed')]
    dfm['days'] = d - dfm['cl_fst_serv_dt_x']
    dfs.append(dfm)

In [ ]:
df_all = pd.concat(dfs)

In [ ]:
df_all.head()

In [ ]:
df_all.to_csv('data/confirmed_members/post_cov_w_days.csv', index=False)

In [ ]:
df_30 = df_all[df_all['days'].dt.days<=30]

In [ ]:
len(df_30['clm_patient_id'].unique())

In [ ]:
len(df_30['clm_patient_id'].unique())/len(mems)

In [ ]:
len(mems)

In [ ]:
len(df_all)

In [ ]:
df1 = df_c_m_c[~(df_c_m_c.duplicated(['cov_19']) & df_c_m_c['cov_19'].eq('Confirmed'))]

In [ ]:
df1['clm_patient_id'].value_counts()

In [ ]:
df1['cov_19'].value_counts()

In [ ]:
len(df_c_m_c[df_c_m_c['clm_patient_id']=='86011715918M19520319'])

In [ ]:
len(df1[df1['clm_patient_id']=='86011715918M19520319'])

In [ ]:
len(df1.drop_duplicates('clm_patient_id'))

In [ ]:
df2 = df1[~df1['cov_19'].eq('Confirmed').groupby(df1['clm_patient_id']).cumsum().astype(bool)]

In [ ]:
len(df2.drop_duplicates('clm_patient_id'))

In [ ]:
df2['cov_19'].value_counts()

In [ ]:
df2.head()

In [ ]:
cols = ['clm_hc_diag_code1', 'clm_hc_diag_code10',
       'clm_hc_diag_code2', 'clm_hc_diag_code3', 'clm_hc_diag_code4',
       'clm_hc_diag_code5', 'clm_hc_diag_code6', 'clm_hc_diag_code7',
       'clm_hc_diag_code8', 'clm_hc_diag_code9']

In [ ]:
df['']

In [ ]:
df3 = df_all[cols].melt(var_name='columns', value_name='DX').reset_index()

In [ ]:
df3.head()

In [ ]:
d_out = df3['DX'].value_counts().reset_index()

In [ ]:
d_out.head()

In [ ]:
d_out.to_csv('data/confirmed_members/single_dxs.csv', index=False)

In [ ]:
df3.head()

In [ ]:
d_out

In [ ]:
d_out.to_csv('data/confirmed_members/pre-u071.csv', index=False)